In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import lzma
import json
import pandas as pd
from collections import defaultdict
from pathlib import Path

In [3]:
from collections import OrderedDict

1. Carregar o arquivo original do LawInstruct e verificar quantos datapoints estão en inglês.

In [4]:
input_dir = "/content/drive/MyDrive/DOUTORADO/TESE/lawinstruct/english_only_data/deduplicated"

In [5]:
output_dir = "/content/drive/MyDrive/DOUTORADO/TESE/lawinstruct/english_only_data/alpaca_format"

In [6]:
input_path = Path(input_dir)
output_path = Path(output_dir)

In [7]:
def transform_entry(entry, id_counter):
    """Transform the instructions to Alpaca format and adds an ID."""
    return {
        "id": id_counter,
        "instruction": entry["instruction"],
        "input": entry["prompt"],
        "output": entry["answer"]
    }

In [ ]:
# def process_json_xz(input_path, output_path):
#     """Process the compressed JSON file and save transformed data."""
#     # Read the input file
#     with lzma.open(input_path, 'rt', encoding='utf-8') as f:
#         content = f.read()

#     # Try different parsing strategies
#     try:
#         # First try standard JSON parsing
#         data = json.loads(content)
#     except json.JSONDecodeError as e:
#         try:
#             # If that fails, try as JSON Lines (one object per line)
#             data = [json.loads(line) for line in content.splitlines() if line.strip()]
#         except json.JSONDecodeError:
#             raise ValueError("Could not parse the file as either JSON or JSON Lines")

#     # Normalize to list
#     if not isinstance(data, list):
#         data = [data]

#     # Transform all entries
#     transformed = [transform_entry(entry, i+1) for i, entry in enumerate(data)]

#     # Write the output file
#     with lzma.open(output_path, 'wt', encoding='utf-8') as f:
#         json.dump(transformed, f, indent=2, ensure_ascii=False)

In [ ]:
# def batch_transform(input_dir, output_dir):
#     """Process all .json.xz files in a directory with new naming convention"""
#     input_dir = Path(input_dir)
#     output_dir = Path(output_dir)
#     output_dir.mkdir(exist_ok=True, parents=True)  # Create output directory including parents

#     for input_file in input_dir.glob("*.json.xz"):
#         # Add 'idf_scores' before extension (handles multiple dots in filename)
#         new_name = f"{input_file.stem}_alpaca_format.json.xz"
#         output_file = output_dir / new_name

#         process_json_xz(input_file, output_file)
#         print(f"Processed: {input_file.name} → {output_file.name}")

In [8]:
# batch_transform(input_path, output_path)

In [9]:
def process_file(input_file, output_file):
    try:
        print(f"\nProcessing: {input_file.name}")
        with lzma.open(input_file, 'rt', encoding='utf-8') as f:
            content = f.read()

        try:
            data = json.loads(content)
        except json.JSONDecodeError:
            data = [json.loads(line) for line in content.splitlines() if line.strip()]

        if not isinstance(data, list):
            data = [data]

        transformed = [transform_entry(entry, i+1) for i, entry in enumerate(data)]

        with lzma.open(output_file, 'wt', encoding='utf-8') as f:
            json.dump(transformed, f, indent=2, ensure_ascii=False)

        print(f"Successfully created: {output_file.name}")
        return True
    except Exception as e:
        print(f"Failed to process {input_file.name}: {str(e)}")
        return False

In [10]:
success_count = 0
for input_file in input_path.glob('*.xz'):  # Changed to .xz
    # Generate output filename
    if input_file.suffix == '.xz':
        # Handle both cases: filename.json.xz and filename.xz
        stem = input_file.stem  # Gets 'filename' from 'filename.json.xz'
        if input_file.name.endswith('.json.xz'):
            stem = input_file.name[:-8]  # Remove '.json.xz'
        output_filename = f"{stem}_alpaca_format.json.xz"
    else:
        output_filename = f"{input_file.stem}_alpaca_format.json.xz"

    output_file = output_path / output_filename

    if process_file(input_file, output_file):
        success_count += 1
        print(f"Created: {output_filename}")
    else:
        print(f"Failed: {input_file.name}")


Processing: BVADecisions-bva_decisions_label-train-0.jsonl_english_only_deduplicated.xz
Successfully created: BVADecisions-bva_decisions_label-train-0.jsonl_english_only_deduplicated_alpaca_format.json.xz
Created: BVADecisions-bva_decisions_label-train-0.jsonl_english_only_deduplicated_alpaca_format.json.xz

Processing: CaseBriefs-case_briefs-train-0.jsonl_english_only_deduplicated.xz
Successfully created: CaseBriefs-case_briefs-train-0.jsonl_english_only_deduplicated_alpaca_format.json.xz
Created: CaseBriefs-case_briefs-train-0.jsonl_english_only_deduplicated_alpaca_format.json.xz

Processing: EOIRPrivacy-eoir_privacy-train-0.jsonl_english_only_deduplicated.xz
Successfully created: EOIRPrivacy-eoir_privacy-train-0.jsonl_english_only_deduplicated_alpaca_format.json.xz
Created: EOIRPrivacy-eoir_privacy-train-0.jsonl_english_only_deduplicated_alpaca_format.json.xz

Processing: LawngNli-lawng_nli_entailment-train-0.jsonl_english_only_deduplicated.xz
Successfully created: LawngNli-lawng_n

In [11]:
from google.colab import runtime
runtime.unassign()